# [Redis](https://python.langchain.com/v0.2/docs/integrations/providers/redis/)

In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

## Cache
The Cache wrapper allows for Redis to be used as a remote, low-latency, in-memory cache for LLM prompts and responses.

Standard Cache
The standard cache is the Redis bread & butter of use case in production for both open-source and enterprise users globally.

In [1]:
from langchain.cache import RedisCache

In [5]:
redis_url = "redis://:@localhost:6379/0" # I do"n't set username and password

In [22]:
from langchain.globals import set_llm_cache
import redis

redis_client = redis.Redis.from_url(redis_url)
set_llm_cache(RedisCache(redis_client))

In [23]:
%%time
llm.invoke("Tell me a joke")

CPU times: user 14.2 ms, sys: 4.93 ms, total: 19.1 ms
Wall time: 1.09 s


AIMessage(content="Sure, here's one for you:\n\nWhy don't skeletons fight each other?\n\nThey don't have the guts!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5', 'finish_reason': 'stop', 'logprobs': None}, id='run-53eb429d-e9c4-418f-b965-3994f75a3165-0', usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32})

In [24]:
%%time
llm.invoke("Tell me a joke")

CPU times: user 2.07 ms, sys: 608 μs, total: 2.68 ms
Wall time: 2.36 ms


AIMessage(content="Sure, here's one for you:\n\nWhy don't skeletons fight each other?\n\nThey don't have the guts!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5', 'finish_reason': 'stop', 'logprobs': None}, id='run-53eb429d-e9c4-418f-b965-3994f75a3165-0', usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32})

In [25]:
%%time
llm.invoke("Tell me why sky is blue?")

CPU times: user 14.3 ms, sys: 4.67 ms, total: 18.9 ms
Wall time: 3.29 s


AIMessage(content="The sky appears blue primarily due to a phenomenon called Rayleigh scattering. Here's a simplified explanation:\n\n1. **Scattering of Sunlight**: Sunlight, or white light, is composed of various colors, each with different wavelengths. When sunlight enters Earth's atmosphere, it collides with gas molecules and small particles.\n\n2. **Wavelength Dependence**: Shorter wavelengths of light (blue and violet) are scattered in all directions more efficiently than longer wavelengths (red and yellow). This is because shorter wavelengths interact more with the small particles and gas molecules in the atmosphere.\n\n3. **Human Perception**: Even though violet light is scattered even more than blue light, our eyes are more sensitive to blue light and less so to violet. Additionally, some of the violet light is absorbed by the upper atmosphere.\n\n4. **Resulting Color**: Because blue light is scattered in all directions and our eyes are more sensitive to it, we perceive the sky

In [26]:
%%time
llm.invoke("Tell me why sky is blue?")

CPU times: user 3.29 ms, sys: 1.43 ms, total: 4.72 ms
Wall time: 4.68 ms


AIMessage(content="The sky appears blue primarily due to a phenomenon called Rayleigh scattering. Here's a simplified explanation:\n\n1. **Scattering of Sunlight**: Sunlight, or white light, is composed of various colors, each with different wavelengths. When sunlight enters Earth's atmosphere, it collides with gas molecules and small particles.\n\n2. **Wavelength Dependence**: Shorter wavelengths of light (blue and violet) are scattered in all directions more efficiently than longer wavelengths (red and yellow). This is because shorter wavelengths interact more with the small particles and gas molecules in the atmosphere.\n\n3. **Human Perception**: Even though violet light is scattered even more than blue light, our eyes are more sensitive to blue light and less so to violet. Additionally, some of the violet light is absorbed by the upper atmosphere.\n\n4. **Resulting Color**: Because blue light is scattered in all directions and our eyes are more sensitive to it, we perceive the sky

## Semantic Cache
Semantic caching allows users to retrieve cached prompts based on semantic similarity between the user input and previously cached results. Under the hood it blends Redis as both a cache and a vectorstore.

In [2]:
from langchain.cache import RedisSemanticCache
from langchain_openai import OpenAIEmbeddings

from langchain.globals import set_llm_cache
import redis

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)
# must use DB0
redis_url = "redis://:@localhost:6379/0" # I do"n't set username and password

set_llm_cache(RedisSemanticCache(
    embedding=embeddings,
    redis_url=redis_url
))

In [3]:
%%time
# 第一次调用时，尚未缓存，所以需要更长的时间
llm.invoke("Tell me a joke")

CPU times: user 217 ms, sys: 46.4 ms, total: 263 ms
Wall time: 814 ms


AIMessage(content="Sure, here's one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 11, 'total_tokens': 30}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5', 'finish_reason': 'stop', 'logprobs': None}, id='run-2c1128af-ee20-4e5f-93df-68ebfb449d03-0', usage_metadata={'input_tokens': 11, 'output_tokens': 19, 'total_tokens': 30})

In [4]:
%%time
# 第二次调用时，虽然不是直接命中，但问题在语义上与原始问题相似，
# 因此使用了缓存的结果！
llm.invoke("Tell me one joke")

CPU times: user 10.7 ms, sys: 4.44 ms, total: 15.1 ms
Wall time: 548 ms


AIMessage(content="Sure, here's one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 11, 'total_tokens': 30}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5', 'finish_reason': 'stop', 'logprobs': None}, id='run-2c1128af-ee20-4e5f-93df-68ebfb449d03-0', usage_metadata={'input_tokens': 11, 'output_tokens': 19, 'total_tokens': 30})